### Bibliotecas

In [1]:

import pandas as pd
import os
import warnings
import streamlit as st
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=pd.errors.DtypeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)


In [2]:
# Definindo o template padrão para todos os gráficos como 'plotly_dark'
pio.templates.default = 'plotly_dark'

### Funções

In [3]:
def ler_arquivos_csv(caminho_pasta, sep=';'):
    """Lê todos os arquivos CSV em uma pasta e concatena em um único DataFrame.

    Args:
        caminho_pasta (str): Caminho da pasta contendo os arquivos CSV.
        sep (str, optional): Separador dos campos nos arquivos CSV. Defaults to ';'.

    Returns:
        pd.DataFrame: DataFrame concatenado com os dados de todos os arquivos.
    """
    # Inicializa um DataFrame vazio
    df = pd.DataFrame()
    # Lista para armazenar os nomes dos arquivos .csv
    files = [f for f in os.listdir(caminho_pasta) if f.endswith('.csv')]
    # Itera por cada arquivo .csv
    for file in files:
        caminho_completo = os.path.join(caminho_pasta, file)
        try:
            df_temp = pd.read_csv(caminho_completo, sep=sep, error_bad_lines=False)
            df = pd.concat([df, df_temp], ignore_index=True)
        except (pd.errors.ParserError, FileNotFoundError, PermissionError) as e:
            # Você pode optar por logar essas exceções em um arquivo ou utilizar um framework de logging
            pass  # Ignora as exceções por enquanto

    return df


### Importar os Datasets

In [4]:
# Relação dos caminhos onde cada dataset esta armazenado 

caminho_despesas = './dados/despesas'
caminho_proposicoes = './dados/proposicoes'
caminho_proposicoes_autores = './dados/proposicoes_autores'
caminho_proposicoes_classificacoes = './dados/proposicoes_classificacoes'
caminho_votacoes = './dados/votacoes'
caminho_app_web = './dados/app_web'

In [5]:
df_despesas_ano = ler_arquivos_csv(caminho_despesas)
df_deputados = pd.read_csv('./dados/deputados.csv', sep=';')
df_deputados_profissao = pd.read_csv('./dados/deputadosProfissoes.csv', sep=';')
df_votacoes_deputados = ler_arquivos_csv(caminho_votacoes)

In [6]:
# Consultando os tamanhos dos dataframes
print(f'O Dataframe df_despesas_ano possui {df_despesas_ano.shape[0]} linhas e {df_despesas_ano.shape[1]} colunas')
print(f'O Dataframe df_votações_deputados possui {df_votacoes_deputados.shape[0]} linhas e {df_votacoes_deputados.shape[1]} colunas')
print(f'O Dataframe df_deputados possui {df_deputados.shape[0]} linhas e {df_deputados.shape[1]} colunas')
print(f'O Dataframe df_deputados_profissões possui {df_deputados_profissao.shape[0]} linhas e {df_deputados_profissao.shape[1]} colunas')

O Dataframe df_despesas_ano possui 584636 linhas e 32 colunas
O Dataframe df_votações_deputados possui 1107001 linhas e 12 colunas
O Dataframe df_deputados possui 7848 linhas e 13 colunas
O Dataframe df_deputados_profissões possui 12533 linhas e 5 colunas


### Pré Processamento

In [7]:
# Criando coluna Ano_Mes
df_votacoes_deputados['dataHoraVoto'] = pd.to_datetime(df_votacoes_deputados['dataHoraVoto'])
df_despesas_ano['Ano_Mes'] = pd.to_datetime(df_despesas_ano['numAno'].astype(str) + '/' + df_despesas_ano['numMes'].astype(str))

In [8]:
# Escolhendo as colunas para o dataset e renomeando-as  
colunas_despesas = ['txNomeParlamentar','ideCadastro', 'sgUF', 'sgPartido','numSubCota',
                    'txtDescricao','txtFornecedor', 'txtCNPJCPF', 'datEmissao',
                    'vlrDocumento', 'vlrGlosa', 'vlrLiquido', 'numMes','numAno',
                    'datPagamentoRestituicao', 'vlrRestituicao','nuDeputadoId','Ano_Mes']

novas_colunas_despesas = ['Nome_Parlamentar','ID_Cadastro','UF','Partido','Num_Sub_Cota',
                          'Descricao_txt','Fornecedor_txt','CNPJ_CPF','Data_Emissão',
                          'Vlr_Documento','Vlr_Glosa','Vlr_Liquido','Mes','Ano','Data_Pagto',
                          'Vlr_Pagto','ID_Deputado','Ano_Mes']

colunas_votacoes = ['idVotacao', 'uriVotacao', 'dataHoraVoto', 'voto', 'deputado_id',
                    'deputado_uri', 'deputado_nome', 'deputado_siglaPartido',
                    'deputado_uriPartido', 'deputado_siglaUf', 'deputado_idLegislatura',
                    'deputado_urlFoto']

novas_colunas_votacoes = ['ID_Votacao', 'URI_Votacao', 'Data_Voto', 'Voto', 'ID_Cadastro',
                         'URI_Deputado', 'Nome_Parlamentar', 'Partido','URI_Partido', 'UF_Deputado',
                         'ID_Legislatura','URI_Deputado_Foto']

colunas_profissao = ['uri', 'id', 'dataHora', 'codTipoProfissao', 'titulo']

novas_colunas_profissao = ['URI', 'ID_Cadastro', 'DataHora', 'Cod_Profissao', 'Profissao']

In [9]:
# Padronização das nomenclaturas do grupo de despesas

despesas_dict = {
                'AQUISIÇÃO DE TOKENS E CERTIFICADOS DIGITAIS': 'CERTIFICADOS DIGITAIS',
                'ASSINATURA DE PUBLICAÇÕES': 'ASS. PUBLICAÇÕES',
                'COMBUSTÍVEIS E LUBRIFICANTES.': 'COMBUSTÍVEIS',
                'CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.': 'CONSULTORIAS / PESQUISAS',
                'DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.': 'ATIVIDADE PARLAMENTAR',
                'FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR': 'ALIMENTAÇÃO',
                'HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.': 'HOSPEDAGEM',
                'LOCAÇÃO OU FRETAMENTO DE AERONAVES': 'LOCAÇÃO AERONAVES',
                'LOCAÇÃO OU FRETAMENTO DE EMBARCAÇÕES': 'LOCAÇÃO EMBARCAÇÕES',
                'LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES': 'LOCAÇÃO AUTOMÓVEIS',
                'MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR': 'MANUTENÇÃO ESCRITÓRIO',
                'PARTICIPAÇÃO EM CURSO, PALESTRA OU EVENTO SIMILAR': 'PALESTRAS',
                'PASSAGEM AÉREA - REEMBOLSO': 'PASSAGEM',
                'PASSAGEM AÉREA - RPA': 'PASSAGEM',
                'PASSAGEM AÉREA - SIGEPA': 'PASSAGEM',
                'PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS': 'PASSAGEM',
                'SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.': 'SERVIÇO DE SEGURANÇA',
                'SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO': 'TAXI / PEDÁGIO / OUTROS',
                'SERVIÇOS POSTAIS': 'SERVIÇOS POSTAIS',
                'TELEFONIA': 'TELEFONIA'
}


In [10]:
# Ajustando o dataframe com as novas colunas
df_despesas = df_despesas_ano[colunas_despesas]

df_despesas.rename(columns=dict(zip(colunas_despesas, novas_colunas_despesas)), inplace=True)

df_votacoes_deputados.rename(columns=dict(zip(colunas_votacoes, novas_colunas_votacoes)), inplace=True)

df_deputados_profissao.rename(columns=dict(zip(colunas_profissao, novas_colunas_profissao)), inplace=True)

C:\Users\Gabriel Gonçalves\AppData\Local\Temp\ipykernel_4676\3540501791.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_despesas.rename(columns=dict(zip(colunas_despesas, novas_colunas_despesas)), inplace=True)


In [11]:
# Ajustando e padronizando a variável que contém o nome de cada despesa

#df_despesas['Descricao_txt'] = df_despesas['Descricao_txt'].apply(lambda x: despesas_dict(x))

df_despesas['Descricao_txt'] = df_despesas['Descricao_txt'].map(lambda x: despesas_dict.get(x, x))

C:\Users\Gabriel Gonçalves\AppData\Local\Temp\ipykernel_4676\1359633378.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_despesas['Descricao_txt'] = df_despesas['Descricao_txt'].map(lambda x: despesas_dict.get(x, x))


In [12]:
# Convertendo as colunas de data em datetime
df_despesas['Data_Emissão'] = pd.to_datetime(df_despesas['Data_Emissão'])
df_despesas['Data_Pagto'] = pd.to_datetime(df_despesas['Data_Pagto'])

C:\Users\Gabriel Gonçalves\AppData\Local\Temp\ipykernel_4676\1235657094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_despesas['Data_Emissão'] = pd.to_datetime(df_despesas['Data_Emissão'])
C:\Users\Gabriel Gonçalves\AppData\Local\Temp\ipykernel_4676\1235657094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_despesas['Data_Pagto'] = pd.to_datetime(df_despesas['Data_Pagto'])


In [13]:
df_despesas['Nome_Parlamentar'] = df_despesas['Nome_Parlamentar'].str.upper()
df_votacoes_deputados['Nome_Parlamentar'] = df_votacoes_deputados['Nome_Parlamentar'].str.upper()

C:\Users\Gabriel Gonçalves\AppData\Local\Temp\ipykernel_4676\3572990165.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_despesas['Nome_Parlamentar'] = df_despesas['Nome_Parlamentar'].str.upper()


In [14]:
# Criando um dataframe com dados exclusivos dos deputados + foto

colunas_fotos = ['ID_Cadastro','URI_Deputado_Foto']
df_fotos_deputados = df_votacoes_deputados[colunas_fotos]
df_fotos_deputados = df_fotos_deputados.drop_duplicates()

In [15]:
df_fotos_deputados = df_fotos_deputados.drop_duplicates(subset='ID_Cadastro')
df_deputados_profissao = df_deputados_profissao.drop_duplicates(subset='ID_Cadastro')


In [16]:
# Concatenando os dataframes auxiliares ao principal
df_despesas = df_despesas.merge(df_fotos_deputados,
                                how='left',
                                left_on='ID_Cadastro',
                                right_on='ID_Cadastro')


df_despesas = df_despesas.merge(df_deputados_profissao,
                                how='left',
                                left_on='ID_Cadastro',
                                right_on='ID_Cadastro')


In [17]:
df_despesas.shape

(584636, 23)

### Salvar arquivo para aplicação Web - Dashboard

In [18]:
df_despesas.to_parquet('./dados/app_web/Despesas_final.parquet')

---

### Tabelas / Gráficos

In [18]:
# Dataframe para vizualizações

df_despesas_deputado = df_despesas.groupby(['Ano_Mes','Partido','Nome_Parlamentar','Descricao_txt','Fornecedor_txt'], 
                                            ).agg({'Vlr_Liquido':'sum'}).reset_index()

df_despesas_deputado.head()

,Ano_Mes,Partido,Nome_Parlamentar,Descricao_txt,Fornecedor_txt,Vlr_Liquido
0,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,082 - QI 21 - CASCOL COMBUSTIVEIS PARA VEICULO...,377.25
1,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,AUTO POSTO JK LTDA,275.68
2,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,AUTO POSTO REAL HOTEL E RESTAURANTE LTDA,376.82
3,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,COMECIAL APARECIDA,175.04
4,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,COMERCIAL GARDENIA LTDA,555.72


In [19]:
menor_ano = df_despesas_deputado['Ano_Mes'].min().year
maior_ano = df_despesas_deputado['Ano_Mes'].max().year

---

In [20]:
# Tabela de quantidade de deputados por partido
qtde_parlamentar = df_despesas_deputado.groupby(['Partido'])['Nome_Parlamentar'].nunique().sort_values(ascending=False).reset_index()

In [21]:
fig = px.bar(qtde_parlamentar, x="Partido", y="Nome_Parlamentar", title='Quantidade de deputados por partido')
fig.show()

---

In [22]:
# Gasto mensal por parlamentar

df_despesas_deputado_mes = df_despesas_deputado.groupby(['Ano_Mes', 'Nome_Parlamentar'])['Vlr_Liquido'].sum().reset_index()

In [23]:
# Filtros

deputado = 'DELEGADO PALUMBO'

df_despesas_deputados_mes_filtered = df_despesas_deputado_mes.loc[df_despesas_deputado_mes['Nome_Parlamentar'] == deputado]

In [24]:
fig_gastos_deput = px.line(df_despesas_deputados_mes_filtered,
                           x = 'Ano_Mes',
                           y = 'Vlr_Liquido',
                           title=f'Gasto por deputado entre os anos de {menor_ano} e {maior_ano}')
fig_gastos_deput.show()

---

In [25]:
# Gasto mensal detalhado por tipo de despesa

df_despesas_tipo_mes = df_despesas_deputado.groupby(['Ano_Mes', 'Nome_Parlamentar', 'Descricao_txt' ])['Vlr_Liquido'].sum().reset_index()

In [26]:
ano_menor = '2024'
ano_maior = '2024'

df_despesas_tipo_mes_filtered = df_despesas_tipo_mes.loc[(df_despesas_tipo_mes['Nome_Parlamentar'] == deputado) & 
                                                         (df_despesas_tipo_mes['Ano_Mes'] <= ano_maior) &
                                                         (df_despesas_tipo_mes['Ano_Mes'] >= ano_menor)]

In [27]:
#colunas_2 = ['Descricao_txt', 'Vlr_Liquido']   #'Ano_Mes'

df_despesas_tipo_mes_filtered = df_despesas_tipo_mes_filtered.groupby(['Descricao_txt'])['Vlr_Liquido'].sum().sort_values(ascending=False).reset_index()

In [28]:
df_despesas_tipo_mes_filtered.head(20)

,Descricao_txt,Vlr_Liquido
0,ATIVIDADE PARLAMENTAR,15000.00
1,MANUTENÇÃO ESCRITÓRIO,10869.93
2,SERVIÇO DE SEGURANÇA,6380.40
3,LOCAÇÃO AUTOMÓVEIS,4900.00
4,COMBUSTÍVEIS,1834.99
5,TELEFONIA,825.77
6,ALIMENTAÇÃO,611.45


In [29]:
fig_gastos_tipo = px.bar(df_despesas_tipo_mes_filtered,
                           x = 'Descricao_txt',
                           y = 'Vlr_Liquido',
                           title= f'Gasto detalhado por tipo de despesa entre os anos de {ano_menor} e {ano_maior}')
fig_gastos_tipo.show()

---

In [30]:
# Gasto mensal por partido

df_despesas_partido_ano = df_despesas_deputado.groupby(['Ano_Mes','Partido'], as_index=False).agg({'Vlr_Liquido':'sum'}).sort_values(by='Vlr_Liquido',
                                                                                                                                     ascending=False)

df_despesas_partido_ano.head()

,Ano_Mes,Partido,Vlr_Liquido
516,2023-12-01,PL,4964344.88
432,2023-08-01,PL,4273906.43
474,2023-10-01,PL,4100106.57
495,2023-11-01,PL,4026820.45
453,2023-09-01,PL,3974695.22


In [31]:
# Filtro

ano = '2017'

df_despesas_partido_ano_filtered = df_despesas_partido_ano.loc[df_despesas_partido_ano['Ano_Mes'] == ano]

In [32]:
fig_gastos_partido = px.bar(df_despesas_partido_ano_filtered,
                           x = 'Partido',
                           y = 'Vlr_Liquido',
                           title= f'Gasto por partido no ano de {ano}')
fig_gastos_partido.show()

---

In [33]:
# Gasto mensal por partido e deputado

df_despesas_partido_deputado_ano = df_despesas_deputado.groupby(['Ano_Mes','Partido','Nome_Parlamentar'], as_index=False).agg(
                                {'Vlr_Liquido':'sum'}).sort_values(by='Vlr_Liquido',ascending=False)

df_despesas_partido_deputado_ano.head()

,Ano_Mes,Partido,Nome_Parlamentar,Vlr_Liquido
12147,2023-12-01,PP,AMANDA GENTIL,256959.64
5687,2022-12-01,CIDADANIA,ALEX MANENTE,251770.85
11952,2023-12-01,CIDADANIA,ALEX MANENTE,233053.18
5444,2022-11-01,PSD,EXPEDITO NETTO,197182.59
12382,2023-12-01,REPUBLICANOS,GUSTINHO RIBEIRO,189989.51


In [34]:
# Filtro

partido = 'PT'
ano_1 = '2023'
colunas = ['Ano_Mes', 'Nome_Parlamentar', 'Vlr_Liquido']

df_despesas_partido_deputado_ano_filtered = df_despesas_partido_deputado_ano.loc[(df_despesas_partido_deputado_ano['Partido'] == partido) &
                                                                                 (df_despesas_partido_deputado_ano['Ano_Mes'] == ano_1)]
df_despesas_partido_deputado_ano_filtered = df_despesas_partido_deputado_ano_filtered[colunas]

df_despesas_partido_deputado_ano_filtered.head()


,Ano_Mes,Nome_Parlamentar,Vlr_Liquido
6614,2023-01-01,ZECA DIRCEU,71386.25
6581,2023-01-01,JOSEILDO RAMOS,59381.87
6616,2023-01-01,ZÉ NETO,54630.14
6589,2023-01-01,LUIZIANNE LINS,54325.77
6583,2023-01-01,JOSÉ AIRTON FÉLIX CIRILO,51655.77


In [35]:
fig_gastos_partido_dep = px.bar(df_despesas_partido_deputado_ano_filtered,
                                 x = 'Nome_Parlamentar',
                                 y = 'Vlr_Liquido',
                                title= f'Gasto por partido no ano de {ano} aberto por parlamentar')
fig_gastos_partido_dep.show()

---

In [36]:
df_despesas_fornecedor = df_despesas_deputado.groupby(['Ano_Mes','Partido','Fornecedor_txt'] 
                                        ).agg({'Vlr_Liquido':'sum'}).reset_index()

df_despesas_fornecedor.head()

,Ano_Mes,Partido,Fornecedor_txt,Vlr_Liquido
0,2022-01-01,AVANTE,082 - QI 21 - CASCOL COMBUSTIVEIS PARA VEICULO...,377.25
1,2022-01-01,AVANTE,AC MORAIS COMBUSTIVEIS LTDA,261.30
2,2022-01-01,AVANTE,AUTO LOCADORA SALOME LTDA,6700.00
3,2022-01-01,AVANTE,AUTO POSTO CRIOLO LTDA,150.00
4,2022-01-01,AVANTE,AUTO POSTO DE COMBUSTIVEIS BARREIRINHAS LTDA - ME,202.69


In [37]:
df_despesas_fornecedor_geral = df_despesas_fornecedor.groupby(['Fornecedor_txt']
    ).agg({'Vlr_Liquido':'sum'}).sort_values(by='Vlr_Liquido',ascending=False).reset_index()

df_despesas_fornecedor_geral.head()

,Fornecedor_txt,Vlr_Liquido
0,TAM,65379855.13
1,GOL,29912302.00
2,AZUL,20859458.57
3,PANTANAL VEÍCULOS LTDA,7185829.28
4,Cia Aérea - TAM,5518977.76


In [38]:
df_despesas_partido = df_despesas_partido_ano.groupby(['Partido'], 
            ).agg({'Vlr_Liquido':'sum'}).sort_values(by='Vlr_Liquido',ascending=False).reset_index()

df_despesas_partido.head()

,Partido,Vlr_Liquido
0,PL,99417116.10
1,PT,79528364.44
2,UNIÃO,69401105.59
3,PP,65406388.29
4,REPUBLICANOS,54658136.72


In [39]:
df_despesas_deputado.head()

,Ano_Mes,Partido,Nome_Parlamentar,Descricao_txt,Fornecedor_txt,Vlr_Liquido
0,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,082 - QI 21 - CASCOL COMBUSTIVEIS PARA VEICULO...,377.25
1,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,AUTO POSTO JK LTDA,275.68
2,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,AUTO POSTO REAL HOTEL E RESTAURANTE LTDA,376.82
3,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,COMECIAL APARECIDA,175.04
4,2022-01-01,AVANTE,ANDRÉ JANONES,COMBUSTÍVEIS,COMERCIAL GARDENIA LTDA,555.72


In [45]:
df_despesas_deputado['Partido'].nunique(), df_despesas_deputado['Nome_Parlamentar'].nunique()

(25, 847)

In [46]:
df_despesas_deputado['Vlr_Liquido'].sum()

611521510.1399999